<a href="https://colab.research.google.com/github/shabathink/WholeLearnerBot/blob/main/WholeLearnerBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle

#  GitHub CSV URL
url = 'https://raw.githubusercontent.com/shabathink/WholeLearnerBot/main/learner_inputs.csv'

# Load dataset from raw URL
df = pd.read_csv(url)

# Display the first few rows
df.head()



,input_text,emotion,motivation,expected_bloom
0,I feel stuck,frustration,low,apply
1,I'm improving in this topic,joy,high,evaluate
2,I'm confused again,frustration,low,understand
3,This is going great!,joy,high,create
4,I'm totally lost,sadness,low,remember


In [56]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle

# 1) Load your CSV
url = 'https://raw.githubusercontent.com/shabathink/WholeLearnerBot/main/learner_inputs.csv'
df = pd.read_csv(url)          # assumes columns 'input_text' and 'emotion' exist

# 2) Train emotion classifier
vec = TfidfVectorizer()
X = vec.fit_transform(df['input_text'])
y = df['emotion']
model = LogisticRegression().fit(X, y)

# 3) Ensure the output folder exists
os.makedirs('models', exist_ok=True)

# 4) Save the model + vectorizer
with open('models/emotion_classifier.pkl', 'wb') as f:
    pickle.dump((model, vec), f)

print("Classifier trained and saved to models/emotion_classifier.pkl")


Classifier trained and saved to models/emotion_classifier.pkl


In [57]:
# Train emotion classifier
vec = TfidfVectorizer()
X = vec.fit_transform(df['input_text'])
y = df['emotion']
model = LogisticRegression().fit(X, y)
with open('models/emotion_classifier.pkl', 'wb') as f:
    pickle.dump((model, vec), f)
print("Classifier trained and saved.")

Classifier trained and saved.


In [58]:
# Predict and evaluate
df['predicted'] = model.predict(X)
df['correct'] = df['predicted'] == df['emotion']
accuracy = df['correct'].mean()
print(f"Prediction accuracy: {accuracy:.2f}")

Prediction accuracy: 1.00


In [60]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# 1) Load the sentence-embedding model (downloads the first time)
sef_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2) Load learning objectives from the raw CSV on GitHub
url = 'https://raw.githubusercontent.com/shabathink/WholeLearnerBot/main/learning_objectives.csv'
objectives = pd.read_csv(url)            # make sure it has a column named 'objective'

# 3) SEF-score helper
def compute_sef(response: str, target: str) -> float:
    emb1 = sef_model.encode(response, convert_to_tensor=True)
    emb2 = sef_model.encode(target,    convert_to_tensor=True)
    return float(util.cos_sim(emb1, emb2))

# 4) Example
response = "Let's try solving an example together."
target   = objectives.loc[1, 'objective']   # adjust row/column names if needed

print(f"SEF score: {compute_sef(response, target):.2f}")


SEF score: 0.34


In [61]:
import pandas as pd            # make sure pandas is imported

log_df = pd.DataFrame({
    'user_id':   [1, 2],
    'input_text': ['I feel stuck', 'This is going great!'],
    'emotion':   ['frustration', 'joy'],
    'tier':      [1, 3],
    'response':  ["Let's try an example.",  # ← fixed quotes
                  'Try designing your own version!'],
    'sef_score': [0.89, 0.91]
})

# Make sure the folder exists before writing
# (e.g. Path('results').mkdir(exist_ok=True))
log_df.to_csv('https://github.com/shabathink/WholeLearnerBot/blob/main/logs.csv', index=False)
log_df


,user_id,input_text,emotion,tier,response,sef_score
0,1,I feel stuck,frustration,1,Let's try an example.,0.89
1,2,This is going great!,joy,3,Try designing your own version!,0.91
